In [16]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import copy
import xml.etree.ElementTree as et
import os

# Stop the execution of a cell (call it with "raise StopExecution")
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# General Methods

In [17]:
def getAccuracy(tp, tn, fp, fn):
    return (tp + tn) / (tp + tn + fp + fn)


def getPrecision(tp, fp):
    if tp == 0 and fp == 0:
        return np.nan
    return tp / (tp + fp)


def getRecall(tp, fn):
    if tp == 0 and fn == 0:
        return np.nan
    return tp / (tp + fn)


def getF1(tp, fp, fn):
    if tp == 0 and (fp == 0 or fn == 0): #If precision or recall are NaN
        return np.nan
    return tp / (tp + 0.5 * (fp + fn))


def printResults(tp, tn, fp, fn):
    print('TP = ' + str(tp))
    print('TN = ' + str(tn))
    print('FP = ' + str(fp))
    print('FN = ' + str(fn))
    print('Accuracy = {:.3f}'.format(getAccuracy(tp, tn, fp, fn)))
    print('Precision = {:.3f}'.format(getPrecision(tp, fp)))
    print('Recall = {:.3f}'.format(getRecall(tp, fn)))
    print('F1-score = {:.3f}'.format(getF1(tp, fp, fn)))

# Accuracy on Type of Instance (i.e., Method Name)

In [18]:
def getMethods(string_methods):
    if string_methods == '[]':
        return []
    method_list = string_methods[1:-1].split('), ')
    output = []
    for k, m in enumerate(method_list):
        if k != len(method_list)-1:
            output.append(m + ')')
        else:
            output.append(m)
    return output


def getHotspotMethods(loads, durations, app):
    output_set = set()
    for l in loads:
        for d in durations:
            tree = et.parse('./hotspots/' + str(l) + '-' + str(d) + '/' + str(app) + '/CPU-hot-spots.xml')
            root = tree.getroot()
            for child in root:
                output_set.add(child.attrib['method'].split(' ')[0])
    return output_set
                
                
def getGroundTruth(df, oracle='minClientsMaxDuration', minVotes=0):
    if oracle == 'minClientsMaxDuration':
        return getMethods(df[(df['load']==LOAD_TRUTH) & (df['duration']==DURATION_TRUTH)]['methodsAP'].iloc[0])
    elif oracle == 'vote':
        if minVotes > 0:
            polls = {}
            for load in LOADS:
                for duration in DURATIONS:
                    methodList = getMethods(df[(df['load']==load) & (df['duration']==duration)]['methodsAP'].iloc[0])
                    for method in methodList:
                        if method in polls:
                            polls[method] += 1
                        else:
                            polls[method] = 1
            output = []
            #print(polls)
            for method in polls:
                if polls[method] >= minVotes:
                    output.append(method)
            return output
        else:
            print('Provide the minimum number of votes (minVotes) to consider a method a true positive')
            raise StopExecution


def getTruePositives(df, gt):
    if len(gt) == 0:
        return 0 #If no methods are in gt then there aren't TP
    chk = getMethods(df['methodsAP'].iloc[0])
    count = 0
    for m in gt:
        if m in chk:
            count += 1
    return count    


def getFalsePositives(df, gt):
    chk = getMethods(df['methodsAP'].iloc[0])
    count = 0
    for m in gt:
        if m in chk:
            count += 1 #This counts the TP (i.e., methods in gt and in chk)
    return len(chk) - count #If methods in chk are not in gt then those are FP


def getFalseNegatives(df, gt):
    if len(gt) == 0:
        return 0 #If no methods are in gt then there aren't FN
    chk = getMethods(df['methodsAP'].iloc[0])
    count = 0
    for m in gt:
        if m not in chk:
            count += 1
    return count #If methods in gt are not also in chk then those are FN

Check that the file is available before running the next cell

In [19]:
dfOrig = pd.read_csv('./data/accuracy.csv', sep=';')

In [20]:
warnings.filterwarnings('ignore')

# General parameters
#SYSTEMS = ['petclinic', 'broadleaf', 'webgoat', 'trainticket', 'openmrs']
#SYSTEM_NAMES = ['PetClinic', 'Broadleaf', 'WebGoat', 'TrainTicket', 'OpenMRS']
SYSTEMS = ['cloudstore', 'teastore', 'webgoat', 'trainticket', 'openmrs']
SYSTEM_NAMES = ['CloudStore', 'TeaStore', 'WebGoat', 'TrainTicket', 'OpenMRS']
LOADS = [25, 50, 75, 100]
DURATIONS = [3, 6, 12]
LOAD_TRUTH = 25
DURATION_TRUTH = 12
MIN_VOTES = 6

# Outputs
SYSTEMS_list = []
for system in SYSTEM_NAMES:
    numSubRows = len(LOADS) * len(DURATIONS)
    SYSTEMS_list.append('\multirow{' + str(numSubRows) + '}{*}{\\rotatebox[origin=c]{90}{\\textbf{' + str(system) + '}}}')
    for i in range(numSubRows-1):
        SYSTEMS_list.append('')
LOADS_list = []
for load in LOADS:
    numSubRows = len(DURATIONS)
    LOADS_list.append('\multirow{' + str(numSubRows) + '}{*}{' + str(load) + '}')
    for i in range(numSubRows - 1):
        LOADS_list.append('')
LOADS_list = LOADS_list * len(SYSTEMS)
DURATIONS_list = DURATIONS * len(LOADS) * len(SYSTEMS)
CTH_list_A = []
EP_list_A = []
WCS_list_A = []
BLOB_list_A = []
TOB_list_A = []
EST_list_A = []
EDA_list_A = []
CTH_list_P = []
EP_list_P = []
WCS_list_P = []
BLOB_list_P = []
TOB_list_P = []
EST_list_P = []
EDA_list_P = []
CTH_list_R = []
EP_list_R = []
WCS_list_R = []
BLOB_list_R = []
TOB_list_R = []
EST_list_R = []
EDA_list_R = []
CTH_list_F = []
EP_list_F = []
WCS_list_F = []
BLOB_list_F = []
TOB_list_F = []
EST_list_F = []
EDA_list_F = []

CTH_list_TP = []
CTH_list_TN = []
CTH_list_FP = []
CTH_list_FN = []
EP_list_TP = []
EP_list_TN = []
EP_list_FP = []
EP_list_FN = []
WCS_list_TP = []
WCS_list_TN = []
WCS_list_FP = []
WCS_list_FN = []
BLOB_list_TP = []
BLOB_list_TN = []
BLOB_list_FP = []
BLOB_list_FN = []
TOB_list_TP = []
TOB_list_TN = []
TOB_list_FP = []
TOB_list_FN = []
EST_list_TP = []
EST_list_TN = []
EST_list_FP = []
EST_list_FN = []
EDA_list_TP = []
EDA_list_TN = []
EDA_list_FP = []
EDA_list_FN = []

In [21]:
AP = 'CTH'

COUNT_TH = 5
CPU_TH = 10
OPTION = 'average'

for SYSTEM in SYSTEMS:
    print('### ' + SYSTEM + ' ###')
    HOTSPOTS = getHotspotMethods(LOADS, DURATIONS, SYSTEM)
    dfTmp = dfOrig[(dfOrig['type']==AP) &
               (dfOrig['system']==SYSTEM) &
               (dfOrig['countTh']==COUNT_TH) &
               (dfOrig['cpuTh']==CPU_TH) &
               (dfOrig['option']==OPTION)
              ]
    gt_methods = getGroundTruth(dfTmp, oracle='vote', minVotes=MIN_VOTES)
    for LOAD in LOADS:
        for DURATION in DURATIONS:
            print('=== ' + str(LOAD) + ' - ' + str(DURATION) + ' ===')
            dfTmp2 = dfTmp[(dfTmp['load']==LOAD) &
                          (dfTmp['duration']==DURATION)
                         ]
            tp = getTruePositives(dfTmp2, gt_methods)
            fp = getFalsePositives(dfTmp2, gt_methods)
            fn = getFalseNegatives(dfTmp2, gt_methods)
            tn = len(HOTSPOTS) - tp - fp - fn
            printResults(tp, tn, fp, fn)
            CTH_list_A.append(getAccuracy(tp, tn, fp, fn))
            CTH_list_P.append(getPrecision(tp, fp))
            CTH_list_R.append(getRecall(tp, fn))
            CTH_list_F.append(getF1(tp, fp, fn))
            
            CTH_list_TP.append(tp)
            CTH_list_TN.append(tn)
            CTH_list_FP.append(fp)
            CTH_list_FN.append(fn)
    print()

### cloudstore ###
=== 25 - 3 ===
TP = 2
TN = 27
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 25 - 6 ===
TP = 2
TN = 27
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 25 - 12 ===
TP = 2
TN = 27
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 50 - 3 ===
TP = 2
TN = 27
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 50 - 6 ===
TP = 2
TN = 27
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 50 - 12 ===
TP = 2
TN = 27
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 75 - 3 ===
TP = 2
TN = 27
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 75 - 6 ===
TP = 2
TN = 27
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 75 - 12 ===
TP = 2
TN = 27
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.

In [22]:
AP = 'EP'

TIME_TH = 5

for SYSTEM in SYSTEMS:
    print('### ' + SYSTEM + ' ###')
    HOTSPOTS = getHotspotMethods(LOADS, DURATIONS, SYSTEM)
    dfTmp = dfOrig[(dfOrig['type']==AP) &
               (dfOrig['system']==SYSTEM) &
               (dfOrig['methodTimeTh']==TIME_TH)
              ]
    gt_methods = getGroundTruth(dfTmp, oracle='vote', minVotes=MIN_VOTES)
    for LOAD in LOADS:
        for DURATION in DURATIONS:
            print('=== ' + str(LOAD) + ' - ' + str(DURATION) + ' ===')
            dfTmp2 = dfTmp[(dfTmp['load']==LOAD) &
                          (dfTmp['duration']==DURATION)
                         ]
            tp = getTruePositives(dfTmp2, gt_methods)
            fp = getFalsePositives(dfTmp2, gt_methods)
            fn = getFalseNegatives(dfTmp2, gt_methods)
            tn = len(HOTSPOTS) - tp - fp - fn
            printResults(tp, tn, fp, fn)
            EP_list_A.append(getAccuracy(tp, tn, fp, fn))
            EP_list_P.append(getPrecision(tp, fp))
            EP_list_R.append(getRecall(tp, fn))
            EP_list_F.append(getF1(tp, fp, fn))
            
            EP_list_TP.append(tp)
            EP_list_TN.append(tn)
            EP_list_FP.append(fp)
            EP_list_FN.append(fn)
    print()

### cloudstore ###
=== 25 - 3 ===
TP = 7
TN = 22
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 25 - 6 ===
TP = 7
TN = 22
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 25 - 12 ===
TP = 6
TN = 22
FP = 0
FN = 1
Accuracy = 0.966
Precision = 1.000
Recall = 0.857
F1-score = 0.923
=== 50 - 3 ===
TP = 7
TN = 21
FP = 1
FN = 0
Accuracy = 0.966
Precision = 0.875
Recall = 1.000
F1-score = 0.933
=== 50 - 6 ===
TP = 6
TN = 22
FP = 0
FN = 1
Accuracy = 0.966
Precision = 1.000
Recall = 0.857
F1-score = 0.923
=== 50 - 12 ===
TP = 7
TN = 21
FP = 1
FN = 0
Accuracy = 0.966
Precision = 0.875
Recall = 1.000
F1-score = 0.933
=== 75 - 3 ===
TP = 6
TN = 22
FP = 0
FN = 1
Accuracy = 0.966
Precision = 1.000
Recall = 0.857
F1-score = 0.923
=== 75 - 6 ===
TP = 7
TN = 22
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 75 - 12 ===
TP = 6
TN = 22
FP = 0
FN = 1
Accuracy = 0.966
Precision = 1.000
Recall = 0.

In [23]:
AP = 'WCS'

MEM_TH = 5

for SYSTEM in SYSTEMS:
    print('### ' + SYSTEM + ' ###')
    HOTSPOTS = getHotspotMethods(LOADS, DURATIONS, SYSTEM)
    dfTmp = dfOrig[(dfOrig['type']==AP) &
               (dfOrig['system']==SYSTEM) &
               (dfOrig['memoryUsageTh']==MEM_TH)
              ]
    gt_methods = getGroundTruth(dfTmp, oracle='vote', minVotes=MIN_VOTES)
    for LOAD in LOADS:
        for DURATION in DURATIONS:
            print('=== ' + str(LOAD) + ' - ' + str(DURATION) + ' ===')
            dfTmp2 = dfTmp[(dfTmp['load']==LOAD) &
                          (dfTmp['duration']==DURATION)
                         ]
            tp = getTruePositives(dfTmp2, gt_methods)
            fp = getFalsePositives(dfTmp2, gt_methods)
            fn = getFalseNegatives(dfTmp2, gt_methods)
            tn = len(HOTSPOTS) - tp - fp - fn
            printResults(tp, tn, fp, fn)
            WCS_list_A.append(getAccuracy(tp, tn, fp, fn))
            WCS_list_P.append(getPrecision(tp, fp))
            WCS_list_R.append(getRecall(tp, fn))
            WCS_list_F.append(getF1(tp, fp, fn))
            
            WCS_list_TP.append(tp)
            WCS_list_TN.append(tn)
            WCS_list_FP.append(fp)
            WCS_list_FN.append(fn)
    print()

### cloudstore ###
=== 25 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 25 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 25 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 6 ===
TP = 0
TN = 27
FP = 2
FN = 0
Accuracy = 0.931
Precision = 0.000
Recall = nan
F1-score = nan
=== 75 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 100 - 3 ===
TP = 0
TN = 29


In [24]:
AP = 'BLOB'

MSG_TH = 5
CPU_TH = 10
HEAP_TH = 10

for SYSTEM in SYSTEMS:
    print('### ' + SYSTEM + ' ###')
    HOTSPOTS = getHotspotMethods(LOADS, DURATIONS, SYSTEM)
    dfTmp = dfOrig[(dfOrig['type']==AP) &
               (dfOrig['system']==SYSTEM) &
               (dfOrig['numMsgs']==MSG_TH) &
               (dfOrig['cpuTh']==CPU_TH) &
               (dfOrig['heapTh']==HEAP_TH)
              ]
    gt_methods = getGroundTruth(dfTmp, oracle='vote', minVotes=MIN_VOTES)
    for LOAD in LOADS:
        for DURATION in DURATIONS:
            print('=== ' + str(LOAD) + ' - ' + str(DURATION) + ' ===')
            dfTmp2 = dfTmp[(dfTmp['load']==LOAD) &
                          (dfTmp['duration']==DURATION)
                         ]
            tp = getTruePositives(dfTmp2, gt_methods)
            fp = getFalsePositives(dfTmp2, gt_methods)
            fn = getFalseNegatives(dfTmp2, gt_methods)
            tn = len(HOTSPOTS) - tp - fp - fn
            printResults(tp, tn, fp, fn)
            BLOB_list_A.append(getAccuracy(tp, tn, fp, fn))
            BLOB_list_P.append(getPrecision(tp, fp))
            BLOB_list_R.append(getRecall(tp, fn))
            BLOB_list_F.append(getF1(tp, fp, fn))
            
            BLOB_list_TP.append(tp)
            BLOB_list_TN.append(tn)
            BLOB_list_FP.append(fp)
            BLOB_list_FN.append(fn)
    print()

### cloudstore ###
=== 25 - 3 ===
TP = 8
TN = 21
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 25 - 6 ===
TP = 8
TN = 21
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 25 - 12 ===
TP = 7
TN = 21
FP = 0
FN = 1
Accuracy = 0.966
Precision = 1.000
Recall = 0.875
F1-score = 0.933
=== 50 - 3 ===
TP = 8
TN = 21
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 50 - 6 ===
TP = 7
TN = 21
FP = 0
FN = 1
Accuracy = 0.966
Precision = 1.000
Recall = 0.875
F1-score = 0.933
=== 50 - 12 ===
TP = 8
TN = 20
FP = 1
FN = 0
Accuracy = 0.966
Precision = 0.889
Recall = 1.000
F1-score = 0.941
=== 75 - 3 ===
TP = 7
TN = 21
FP = 0
FN = 1
Accuracy = 0.966
Precision = 1.000
Recall = 0.875
F1-score = 0.933
=== 75 - 6 ===
TP = 8
TN = 21
FP = 0
FN = 0
Accuracy = 1.000
Precision = 1.000
Recall = 1.000
F1-score = 1.000
=== 75 - 12 ===
TP = 7
TN = 21
FP = 0
FN = 1
Accuracy = 0.966
Precision = 1.000
Recall = 0.

In [25]:
AP = 'TOB'

TIME_TH = 5

for SYSTEM in SYSTEMS:
    print('### ' + SYSTEM + ' ###')
    HOTSPOTS = getHotspotMethods(LOADS, DURATIONS, SYSTEM)
    dfTmp = dfOrig[(dfOrig['type']==AP) &
               (dfOrig['system']==SYSTEM) &
               (dfOrig['methodTimeTh']==TIME_TH)
              ]
    gt_methods = getGroundTruth(dfTmp, oracle='vote', minVotes=MIN_VOTES)
    for LOAD in LOADS:
        for DURATION in DURATIONS:
            print('=== ' + str(LOAD) + ' - ' + str(DURATION) + ' ===')
            dfTmp2 = dfTmp[(dfTmp['load']==LOAD) &
                          (dfTmp['duration']==DURATION)
                         ]
            tp = getTruePositives(dfTmp2, gt_methods)
            fp = getFalsePositives(dfTmp2, gt_methods)
            fn = getFalseNegatives(dfTmp2, gt_methods)
            tn = len(HOTSPOTS) - tp - fp - fn
            printResults(tp, tn, fp, fn)
            TOB_list_A.append(getAccuracy(tp, tn, fp, fn))
            TOB_list_P.append(getPrecision(tp, fp))
            TOB_list_R.append(getRecall(tp, fn))
            TOB_list_F.append(getF1(tp, fp, fn))
            
            TOB_list_TP.append(tp)
            TOB_list_TN.append(tn)
            TOB_list_FP.append(fp)
            TOB_list_FN.append(fn)
    print()

### cloudstore ###
=== 25 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 25 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 25 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 100 - 3 ===
TP = 0
TN = 29
FP

In [26]:
AP = 'EST'

MSGS_TH = 5

for SYSTEM in SYSTEMS:
    print('### ' + SYSTEM + ' ###')
    HOTSPOTS = getHotspotMethods(LOADS, DURATIONS, SYSTEM)
    dfTmp = dfOrig[(dfOrig['type']==AP) &
               (dfOrig['system']==SYSTEM) &
               (dfOrig['numMsgs']==MSGS_TH)
              ]
    gt_methods = getGroundTruth(dfTmp, oracle='vote', minVotes=MIN_VOTES)
    for LOAD in LOADS:
        for DURATION in DURATIONS:
            print('=== ' + str(LOAD) + ' - ' + str(DURATION) + ' ===')
            dfTmp2 = dfTmp[(dfTmp['load']==LOAD) &
                          (dfTmp['duration']==DURATION)
                         ]
            tp = getTruePositives(dfTmp2, gt_methods)
            fp = getFalsePositives(dfTmp2, gt_methods)
            fn = getFalseNegatives(dfTmp2, gt_methods)
            tn = len(HOTSPOTS) - tp - fp - fn
            printResults(tp, tn, fp, fn)
            EST_list_A.append(getAccuracy(tp, tn, fp, fn))
            EST_list_P.append(getPrecision(tp, fp))
            EST_list_R.append(getRecall(tp, fn))
            EST_list_F.append(getF1(tp, fp, fn))
            
            EST_list_TP.append(tp)
            EST_list_TN.append(tn)
            EST_list_FP.append(fp)
            EST_list_FN.append(fn)
    print()

### cloudstore ###
=== 25 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 25 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 25 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 100 - 3 ===
TP = 0
TN = 29
FP

In [27]:
AP = 'EDA'

OBJS_TH = 5
HEAP_TH = 10

for SYSTEM in SYSTEMS:
    print('### ' + SYSTEM + ' ###')
    HOTSPOTS = getHotspotMethods(LOADS, DURATIONS, SYSTEM)
    dfTmp = dfOrig[(dfOrig['type']==AP) &
               (dfOrig['system']==SYSTEM) &
               (dfOrig['numGCedObjsTh']==OBJS_TH) &
               (dfOrig['heapTh']==HEAP_TH)
              ]
    gt_methods = getGroundTruth(dfTmp, oracle='vote', minVotes=MIN_VOTES)
    for LOAD in LOADS:
        for DURATION in DURATIONS:
            print('=== ' + str(LOAD) + ' - ' + str(DURATION) + ' ===')
            dfTmp2 = dfTmp[(dfTmp['load']==LOAD) &
                          (dfTmp['duration']==DURATION)
                         ]
            tp = getTruePositives(dfTmp2, gt_methods)
            fp = getFalsePositives(dfTmp2, gt_methods)
            fn = getFalseNegatives(dfTmp2, gt_methods)
            tn = len(HOTSPOTS) - tp - fp - fn
            printResults(tp, tn, fp, fn)
            EDA_list_A.append(getAccuracy(tp, tn, fp, fn))
            EDA_list_P.append(getPrecision(tp, fp))
            EDA_list_R.append(getRecall(tp, fn))
            EDA_list_F.append(getF1(tp, fp, fn))
            
            EDA_list_TP.append(tp)
            EDA_list_TN.append(tn)
            EDA_list_FP.append(fp)
            EDA_list_FN.append(fn)
    print()

### cloudstore ###
=== 25 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 25 - 6 ===
TP = 0
TN = 27
FP = 2
FN = 0
Accuracy = 0.931
Precision = 0.000
Recall = nan
F1-score = nan
=== 25 - 12 ===
TP = 0
TN = 27
FP = 2
FN = 0
Accuracy = 0.931
Precision = 0.000
Recall = nan
F1-score = nan
=== 50 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 50 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 3 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 6 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 75 - 12 ===
TP = 0
TN = 29
FP = 0
FN = 0
Accuracy = 1.000
Precision = nan
Recall = nan
F1-score = nan
=== 100 - 3 ===
TP = 0
TN = 2

In [13]:
warnings.filterwarnings('ignore')

# General parameters
SYSTEMS = ['cloudstore', 'teastore', 'webgoat', 'trainticket', 'openmrs']
SYSTEM_NAMES = ['CloudStore', 'TeaStore', 'WebGoat', 'TrainTicket', 'OpenMRS']
MEASURES = ['A', 'P', 'R', 'F1']

# Outputs
SYSTEMS_list = []
for system in SYSTEM_NAMES:
    numSubRows = len(MEASURES)
    SYSTEMS_list.append('\multirow{' + str(numSubRows) + '}{*}{\\rotatebox[origin=c]{0}{\\textbf{' + str(system) + '}}}')
    for i in range(numSubRows-1):
        SYSTEMS_list.append('')
MEASURES_list = MEASURES * len(SYSTEMS)

# Generate the Accuracy Table

In [14]:
STEP = len(LOADS) * len(DURATIONS)

CTH_sys_A = [np.mean(CTH_list_A[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
CTH_sys_P = [np.mean(CTH_list_P[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
CTH_sys_R = [np.mean(CTH_list_R[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
CTH_sys_F = [2 * CTH_sys_P[x] * CTH_sys_R[x] / (CTH_sys_P[x] + CTH_sys_R[x]) for x in range(len(CTH_sys_P))]

EP_sys_A = [np.mean(EP_list_A[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
EP_sys_P = [np.mean(EP_list_P[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
EP_sys_R = [np.mean(EP_list_R[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
EP_sys_F = [2 * EP_sys_P[x] * EP_sys_R[x] / (EP_sys_P[x] + EP_sys_R[x]) for x in range(len(EP_sys_P))]

WCS_sys_A = [np.mean(WCS_list_A[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
WCS_sys_P = [np.mean(WCS_list_P[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
WCS_sys_R = [np.mean(WCS_list_R[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
WCS_sys_F = [2 * WCS_sys_P[x] * WCS_sys_R[x] / (WCS_sys_P[x] + WCS_sys_R[x]) for x in range(len(WCS_sys_P))]

BLOB_sys_A = [np.mean(BLOB_list_A[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
BLOB_sys_P = [np.mean(BLOB_list_P[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
BLOB_sys_R = [np.mean(BLOB_list_R[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
BLOB_sys_F = [2 * BLOB_sys_P[x] * BLOB_sys_R[x] / (BLOB_sys_P[x] + BLOB_sys_R[x]) for x in range(len(BLOB_sys_P))]

TOB_sys_A = [np.mean(TOB_list_A[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
TOB_sys_P = [np.mean(TOB_list_P[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
TOB_sys_R = [np.mean(TOB_list_R[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
TOB_sys_F = [2 * TOB_sys_P[x] * TOB_sys_R[x] / (TOB_sys_P[x] + TOB_sys_R[x]) for x in range(len(TOB_sys_P))]

EST_sys_A = [np.mean(EST_list_A[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
EST_sys_P = [np.mean(EST_list_P[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
EST_sys_R = [np.mean(EST_list_R[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
EST_sys_F = [2 * EST_sys_P[x] * EST_sys_R[x] / (EST_sys_P[x] + EST_sys_R[x]) for x in range(len(EST_sys_P))]

EDA_sys_A = [np.mean(EDA_list_A[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
EDA_sys_P = [np.mean(EDA_list_P[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
EDA_sys_R = [np.mean(EDA_list_R[x*STEP:(x+1)*STEP]) for x in range(len(SYSTEMS))]
EDA_sys_F = [2 * EDA_sys_P[x] * EDA_sys_R[x] / (EDA_sys_P[x] + EDA_sys_R[x]) for x in range(len(EDA_sys_P))]

In [15]:
with open('acc_table.tex', 'w') as f:
    f.write('\\begin{table}[t] \n')
    f.write('\centering \n')
    f.write('\caption{Accuracy} \n')
    f.write('\label{tab:acc} \n')
    f.write('\\resizebox{\linewidth}{!}{ \n')
    f.write('\\begin{tabular}{rr|rrrrrrr} \n')
    f.write('\\toprule \n')
    f.write('&  &  &  &  &  &  &  & \\\\ \n')
    f.write('&  & \emph{CTH} & \emph{EP} & \emph{WCS} & \emph{Blob} & \emph{ToB} & \emph{EST} & \emph{EDA} \\\\ \n')
    f.write('\midrule \n')
    idx = 0
    HEADERS_list = []
    for s, meas in zip (SYSTEMS_list, MEASURES_list):
        HEADERS_list.append(str(s) + ' & ' + str(meas))
    idx = 0
    idx2 = 0
    while idx < len(HEADERS_list):
        f.write(HEADERS_list[idx] + ' & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\ \n'.format(CTH_sys_A[idx2], EP_sys_A[idx2], WCS_sys_A[idx2], BLOB_sys_A[idx2], TOB_sys_A[idx2], EST_sys_A[idx2], EDA_sys_A[idx2]))
        idx += 1
        f.write(HEADERS_list[idx] + ' & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\ \n'.format(CTH_sys_P[idx2], EP_sys_P[idx2], WCS_sys_P[idx2], BLOB_sys_P[idx2], TOB_sys_P[idx2], EST_sys_P[idx2], EDA_sys_P[idx2]))
        idx += 1
        f.write(HEADERS_list[idx] + ' & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\ \n'.format(CTH_sys_R[idx2], EP_sys_R[idx2], WCS_sys_R[idx2], BLOB_sys_R[idx2], TOB_sys_R[idx2], EST_sys_R[idx2], EDA_sys_R[idx2]))
        idx += 1
        f.write(HEADERS_list[idx] + ' & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\ \n'.format(CTH_sys_F[idx2], EP_sys_F[idx2], WCS_sys_F[idx2], BLOB_sys_F[idx2], TOB_sys_F[idx2], EST_sys_F[idx2], EDA_sys_F[idx2]))
        idx += 1
        idx2 += 1
        f.write('\midrule \n')
    f.write('\midrule \n')
    f.write('\\multirow{4}{*}{\\rotatebox[origin=c]{0}{\\textbf{Avg.}}} & A' + ' & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\ \n'.format(np.nanmean(CTH_sys_A), np.nanmean(EP_sys_A), np.nanmean(WCS_sys_A), np.nanmean(BLOB_sys_A), np.nanmean(TOB_sys_A), np.nanmean(EST_sys_A), np.nanmean(EDA_sys_A)))
    f.write('& P & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\ \n'.format(np.nanmean(CTH_sys_P), np.nanmean(EP_sys_P), np.nanmean(WCS_sys_P), np.nanmean(BLOB_sys_P), np.nanmean(TOB_sys_P), np.nanmean(EST_sys_P), np.nanmean(EDA_sys_P)))
    f.write('& R & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\ \n'.format(np.nanmean(CTH_sys_R), np.nanmean(EP_sys_R), np.nanmean(WCS_sys_R), np.nanmean(BLOB_sys_R), np.nanmean(TOB_sys_R), np.nanmean(EST_sys_R), np.nanmean(EDA_sys_R)))
    f.write('& F1 & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\ \n'.format(np.nanmean(CTH_sys_F), np.nanmean(EP_sys_F), np.nanmean(WCS_sys_F), np.nanmean(BLOB_sys_F), np.nanmean(TOB_sys_F), np.nanmean(EST_sys_F), np.nanmean(EDA_sys_F)))
    f.write('\midrule \n')
    f.write('\end{tabular}} \n')
    f.write('\end{table} \n')

# Generate the Confusion Matrix Table

In [40]:
conf_mats_dict = {
    'CTH' : [CTH_list_TP, CTH_list_TN, CTH_list_FP, CTH_list_FN],
    'EP' : [EP_list_TP, EP_list_TN, EP_list_FP, EP_list_FN],
    'WCS' : [WCS_list_TP, WCS_list_TN, WCS_list_FP, WCS_list_FN],
    'BLOB' : [BLOB_list_TP, BLOB_list_TN, BLOB_list_FP, BLOB_list_FN],
    'TOB' : [TOB_list_TP, TOB_list_TN, TOB_list_FP, TOB_list_FN],
    'EST' : [EST_list_TP, EST_list_TN, EST_list_FP, EST_list_FN],
    'EDA' : [EDA_list_TP, EDA_list_TN, EDA_list_FP, EDA_list_FN],
}

In [50]:
APs = ['CTH', 'EP', 'WCS', 'BLOB', 'TOB', 'EST', 'EDA']
MAX_VALS = 12 # Loads * Durations

for ap in APs:
    with open(ap.lower() + '_conf_mat.tex', 'w') as f:
        f.write('\\begin{table} \n')
        f.write('\centering \n')
        f.write('\scriptsize \n')
        f.write('\caption{Confusion matrix of all considered configurations and applications when detecting the ' + ap + ' antipattern.} \n')
        f.write('\label{tab:confusion-matrix-' + ap.lower() + '} \n')
        f.write('\\begin{adjustbox}{angle=90} \n')
        f.write('\\begin{tabular}{rrr|rrrrrrrrrrrr} \n')
        f.write('& & & 25-3 & 25-6 & 25-12 & 50-3 & 50-6 & 50-12 & 75-3 & 75-6 & 75-12 & 100-3 & 100-6 & 100-12 \\\\ \n')
        for idx, system in enumerate(SYSTEM_NAMES):
            f.write('\\midrule \n')
            f.write('\\multicolumn{1}{c}{\\multirow{5}{*}{\\rotatebox[origin=c]{270}{\\textbf{' + system + '}}}} & \\multicolumn{1}{c}{\\multirow{5}{*}{ \n')
            f.write('\\setlength\\tabcolsep{4pt} \n')
            f.write('\\begin{tabular}{|c|c| } \n')
            f.write('\\hline \n')
            f.write('TP & TN \\\\ \n')
            f.write('\\hline \n')
            f.write('FP & FN \\\\ \n')
            f.write('\\hline \n')
            f.write('\\end{tabular}}} &  & \\multirow{5}{*}{\\begin{tabular}{|c|c| } \n')
            count = MAX_VALS
            for tp, tn, fp, fn in zip(conf_mats_dict[ap][0][MAX_VALS*idx:MAX_VALS*(idx+1)], conf_mats_dict[ap][1][MAX_VALS*idx:MAX_VALS*(idx+1)], conf_mats_dict[ap][2][MAX_VALS*idx:MAX_VALS*(idx+1)], conf_mats_dict[ap][3][MAX_VALS*idx:MAX_VALS*(idx+1)]):                
                f.write('\\hline \n')
                f.write(str(tp) + ' & ' + str(tn) + ' \\\\ \n')
                f.write('\\hline \n')
                f.write(str(fp) + ' & ' + str(fn) + ' \\\\ \n')
                f.write('\\hline \n')
                count -= 1
                if count > 0:
                    f.write('\\end{tabular}} & \\multirow{5}{*}{\\begin{tabular}{|c|c| } \n')
            f.write('\\end{tabular}} \\\\ \n')
            for i in range(4):
                f.write('& & & &  &  &  &  &  &  &  &  &  &  & \\\\ \n')
        f.write('\\midrule \n') 
        f.write('\\end{tabular} \n')
        f.write('\\end{adjustbox} \n')
        f.write('\\end{table} \n')

In [31]:
'BLOB'.lower()

'blob'

In [36]:
len(conf_mats_dict['BLOB'][0]) // 7

8

In [39]:
conf_mats_dict['BLOB'][0][12*1:12*2]

[6, 6, 6, 6, 6, 6, 5, 6, 5, 6, 5, 5]